In [2]:
# 1. Importação de bibliotecas

import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

In [3]:
# 2. Configuração do webdriver

options = Options()
# options.add_argument('--headless') # Roda o script sem abrir o navegador
options.add_argument('window-size=1200,700')

In [4]:
# 3. Acesso ao website

url = 'https://www.politicos.org.br/'
navegador = webdriver.Chrome(options=options)
navegador.get(url)

### **Mapeação do website**

In [5]:
# 1. Acessar a aba do ranking

sleep(3)
aba_ranking = navegador.find_element(By.XPATH, '//a[@href="/Ranking"][1]')
aba_ranking.click()

In [6]:
# 2. Clica no botão ver mais até aparecerem todos os políticos

def clicar_ver_mais():
    ver_mais_count = 0
    while True:
        try:
            # Clica no botão ver mais até a última página
            txt_ultimo_deputado = navegador.find_element(By.XPATH, '(//div[@class="styled__ContentTop-sc-18d0wdx-15 hDQuAe"])[last()]/h2[last()]').text
            navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            btn_ver_mais = navegador.find_element(By.XPATH, '//h3[contains(text(),"Ver mais")]')
            btn_ver_mais.click()
            sleep(1)
            txt_ultimo_deputado_2 = navegador.find_element(By.XPATH, '(//div[@class="styled__ContentTop-sc-18d0wdx-15 hDQuAe"])[last()]/h2[last()]').text
            if txt_ultimo_deputado == txt_ultimo_deputado_2:
                ver_mais_count += 1
                sleep(3)
                if ver_mais_count == 5:
                    print("Chegou ao fim da página do Ranking")
                    break
            continue
        except:
            sleep(2)
            continue

clicar_ver_mais()

Chegou ao fim da página do Ranking


In [7]:
# 3. Extrair os dados do político

page_content = navegador.page_source # Extrai a page_source, que é todo o conteúdo da página
site = BeautifulSoup(page_content, 'html.parser') # Faz o parsing da página, ou seja, estrutura o conteúdo da página em HTML
politicos = site.findAll('button', attrs={'class': 'styled__PosicaoContainer-sc-18d0wdx-1 ghQSIQ'})

dados_politicos = []
for politico in politicos:
    politico_ranking = politico.find('h2').text
    politico_nome = politico.find('h2', attrs={'class': 'styled__PosicaoNome-sc-18d0wdx-9 cSQpXh d-md-none'}).text
    politico_cargo = politico.find('h3').text
    politico_partido = politico.find('span').text
    politico_pontuacao = politico.find('div', attrs={'class': 'styled__PoliticosPontuacao-sc-18d0wdx-10 kMEHCY'}).text
    dados_politicos.append([politico_ranking, politico_nome, politico_cargo, politico_partido,politico_pontuacao])

dados_politicos

[['0º', 'Eduardo Girão', 'SenadorPODEMOS - CE', 'PODEMOS - CE', '8,34PONTOS'],
 ['1º',
  'Alex Manente ',
  'Deputado FederalCIDADANIA - SP',
  'CIDADANIA - SP',
  '8,28PONTOS'],
 ['2º',
  'Adriana Ventura ',
  'Deputado FederalNOVO - SP',
  'NOVO - SP',
  '8,10PONTOS'],
 ['3º',
  'Lucas Redecker ',
  'Deputado FederalPSDB - RS',
  'PSDB - RS',
  '8,08PONTOS'],
 ['4º', 'Carlos Portinho', 'SenadorPL - RJ', 'PL - RJ', '8,05PONTOS'],
 ['5º',
  'Marco Bertaiolli ',
  'Deputado FederalPSD - SP',
  'PSD - SP',
  '8,02PONTOS'],
 ['6º',
  'Paula Belmonte ',
  'Deputado FederalCIDADANIA - DF',
  'CIDADANIA - DF',
  '8,00PONTOS'],
 ['7º',
  'Guiga Peixoto ',
  'Deputado FederalPSC - SP',
  'PSC - SP',
  '7,95PONTOS'],
 ['8º',
  'Marcelo Calero ',
  'Deputado FederalPSD - RJ',
  'PSD - RJ',
  '7,93PONTOS'],
 ['9º',
  'Rodrigo Coelho ',
  'Deputado FederalPODEMOS - SC',
  'PODEMOS - SC',
  '7,91PONTOS'],
 ['10º',
  'Oriovisto Guimarães',
  'SenadorPODEMOS - PR',
  'PODEMOS - PR',
  '7,90PONTOS'],


### **Montando a tabela de análise**

In [8]:
# Criando o dataframe a partir dos dados extraídos

df = pd.DataFrame(dados_politicos, columns=['posicao', 'nome', 'cargo', 'partido', 'pontuacao'])
df.head()

,posicao,nome,cargo,partido,pontuacao
0,0º,Eduardo Girão,SenadorPODEMOS - CE,PODEMOS - CE,"8,34PONTOS"
1,1º,Alex Manente,Deputado FederalCIDADANIA - SP,CIDADANIA - SP,"8,28PONTOS"
2,2º,Adriana Ventura,Deputado FederalNOVO - SP,NOVO - SP,"8,10PONTOS"
3,3º,Lucas Redecker,Deputado FederalPSDB - RS,PSDB - RS,"8,08PONTOS"
4,4º,Carlos Portinho,SenadorPL - RJ,PL - RJ,"8,05PONTOS"


In [9]:
# Alterando a coluna posição (removendo o º)

df['posicao'] = df['posicao'].str.replace('º', '')
df.head()

,posicao,nome,cargo,partido,pontuacao
0,0,Eduardo Girão,SenadorPODEMOS - CE,PODEMOS - CE,"8,34PONTOS"
1,1,Alex Manente,Deputado FederalCIDADANIA - SP,CIDADANIA - SP,"8,28PONTOS"
2,2,Adriana Ventura,Deputado FederalNOVO - SP,NOVO - SP,"8,10PONTOS"
3,3,Lucas Redecker,Deputado FederalPSDB - RS,PSDB - RS,"8,08PONTOS"
4,4,Carlos Portinho,SenadorPL - RJ,PL - RJ,"8,05PONTOS"


In [10]:
# Extraindo o estado de cada político

df['estado'] = df['partido'].str.split(' - ', 1, expand=True)[1]
df.head()

C:\Users\gacardoso1\AppData\Local\Temp\ipykernel_11796\426589684.py:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['estado'] = df['partido'].str.split(' - ', 1, expand=True)[1]


,posicao,nome,cargo,partido,pontuacao,estado
0,0,Eduardo Girão,SenadorPODEMOS - CE,PODEMOS - CE,"8,34PONTOS",CE
1,1,Alex Manente,Deputado FederalCIDADANIA - SP,CIDADANIA - SP,"8,28PONTOS",SP
2,2,Adriana Ventura,Deputado FederalNOVO - SP,NOVO - SP,"8,10PONTOS",SP
3,3,Lucas Redecker,Deputado FederalPSDB - RS,PSDB - RS,"8,08PONTOS",RS
4,4,Carlos Portinho,SenadorPL - RJ,PL - RJ,"8,05PONTOS",RJ


In [11]:
# Removendo o nome do partido da coluna cargo
for partido in df['partido']:
    df['cargo'] = df['cargo'].str.replace(partido, "")

df.head()

C:\Users\gacardoso1\AppData\Local\Temp\ipykernel_11796\3703086052.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['cargo'] = df['cargo'].str.replace(partido, "")


,posicao,nome,cargo,partido,pontuacao,estado
0,0,Eduardo Girão,Senador,PODEMOS - CE,"8,34PONTOS",CE
1,1,Alex Manente,Deputado Federal,CIDADANIA - SP,"8,28PONTOS",SP
2,2,Adriana Ventura,Deputado Federal,NOVO - SP,"8,10PONTOS",SP
3,3,Lucas Redecker,Deputado Federal,PSDB - RS,"8,08PONTOS",RS
4,4,Carlos Portinho,Senador,PL - RJ,"8,05PONTOS",RJ


In [12]:
# Removendo o estado da coluna partido

for estado in df['estado']:
    df['partido'] = df['partido'].str.replace(' - ' + estado, '')

df.head()

,posicao,nome,cargo,partido,pontuacao,estado
0,0,Eduardo Girão,Senador,PODEMOS,"8,34PONTOS",CE
1,1,Alex Manente,Deputado Federal,CIDADANIA,"8,28PONTOS",SP
2,2,Adriana Ventura,Deputado Federal,NOVO,"8,10PONTOS",SP
3,3,Lucas Redecker,Deputado Federal,PSDB,"8,08PONTOS",RS
4,4,Carlos Portinho,Senador,PL,"8,05PONTOS",RJ


In [13]:
# Removendo a palavra pontos da coluna pontuação

df['pontuacao'] = df['pontuacao'].str.replace('PONTOS', '')
df.head()

,posicao,nome,cargo,partido,pontuacao,estado
0,0,Eduardo Girão,Senador,PODEMOS,"8,34",CE
1,1,Alex Manente,Deputado Federal,CIDADANIA,"8,28",SP
2,2,Adriana Ventura,Deputado Federal,NOVO,"8,10",SP
3,3,Lucas Redecker,Deputado Federal,PSDB,"8,08",RS
4,4,Carlos Portinho,Senador,PL,"8,05",RJ


In [15]:
# Extraindo posições com erro. Os dois possue esse erro pois são respectivamente o presidente da Câmara e do Senado

a = pd.to_numeric(df.posicao, errors='coerce')
idx = a.isna()
df[idx]

,posicao,nome,cargo,partido,pontuacao,estado
153,Rodrigo Pacheco,Rodrigo Pacheco,Senador,PSD,PresidenteSenado Federal,MG
215,Arthur Lira,Arthur Lira,Deputado Federal,PP,PresidenteCâmara dos Deputados,AL


In [16]:
# Criando a coluna presidente

def checar_presidencia(df):
    if 'Presidente' in df['pontuacao']:
        presidencia = 1
    else:
        presidencia = 0
    return presidencia

df['presidente'] = df.apply(checar_presidencia, axis=1)
df.head()

,posicao,nome,cargo,partido,pontuacao,estado,presidente
0,0,Eduardo Girão,Senador,PODEMOS,"8,34",CE,0
1,1,Alex Manente,Deputado Federal,CIDADANIA,"8,28",SP,0
2,2,Adriana Ventura,Deputado Federal,NOVO,"8,10",SP,0
3,3,Lucas Redecker,Deputado Federal,PSDB,"8,08",RS,0
4,4,Carlos Portinho,Senador,PL,"8,05",RJ,0


In [17]:
df[df['presidente'] == 1].head()

,posicao,nome,cargo,partido,pontuacao,estado,presidente
153,Rodrigo Pacheco,Rodrigo Pacheco,Senador,PSD,PresidenteSenado Federal,MG,1
215,Arthur Lira,Arthur Lira,Deputado Federal,PP,PresidenteCâmara dos Deputados,AL,1


In [18]:
# Adicionando manualmente a posição e a pontuação do Presidente do Senado e do Presidente da Câmara


df.at[153, 'posicao'] = 152
df.at[215, 'posicao'] = 215

df.at[153, 'pontuacao'] = '6,93'
df.at[215, 'pontuacao'] = '6,62'

df[df['presidente'] == 1].head()

,posicao,nome,cargo,partido,pontuacao,estado,presidente
153,152,Rodrigo Pacheco,Senador,PSD,"6,93",MG,1
215,215,Arthur Lira,Deputado Federal,PP,"6,62",AL,1


In [20]:
df['pontuacao'] = df['pontuacao'].str.replace(',','.')
df.head()

,posicao,nome,cargo,partido,pontuacao,estado,presidente
0,0,Eduardo Girão,Senador,PODEMOS,8.34,CE,0
1,1,Alex Manente,Deputado Federal,CIDADANIA,8.28,SP,0
2,2,Adriana Ventura,Deputado Federal,NOVO,8.10,SP,0
3,3,Lucas Redecker,Deputado Federal,PSDB,8.08,RS,0
4,4,Carlos Portinho,Senador,PL,8.05,RJ,0


In [25]:
# Especificando o tipo de cada coluna

df = df.astype({'posicao': 'int',
                'nome': 'str',
                'cargo': 'str',
                'partido': 'str',
                'pontuacao': 'float',
                'estado': 'str',
                'presidente': 'bool'})

df.head()

,posicao,nome,cargo,partido,pontuacao,estado,presidente
0,0,Eduardo Girão,Senador,PODEMOS,8.34,CE,False
1,1,Alex Manente,Deputado Federal,CIDADANIA,8.28,SP,False
2,2,Adriana Ventura,Deputado Federal,NOVO,8.10,SP,False
3,3,Lucas Redecker,Deputado Federal,PSDB,8.08,RS,False
4,4,Carlos Portinho,Senador,PL,8.05,RJ,False


In [26]:
# Salvando a tabela extraída

df.to_csv('dados/dados_ranking_politicos_2022.csv', sep='\t', encoding='utf-8')
